___

## 1. Data Collection
- For this project, we will be using Reddit's API to collect posts from two subreddits: "r/depression" and "r/SuicideWatch"
- We aim to automate as much of this process as possible into neat functions to enable repeatability on the data collection front.
- When collecting data from servers, we will create a randomized delay between requests as a consideration to Reddit's servers and security staff.

In [1]:
import requests
import time
import pandas as pd
from random import randint

### 1.1 Exploring the HTML architecture of the r/depression subreddit page 

In [2]:
#WE WILL SCRAPE THE r/depression AND r/SuicideWatch SUBREDDITS
#LET'S START BY EXPLORING THE HTML INNARDS OF THE FORMER
url_1 = "https://www.reddit.com/r/depression.json"

In [3]:
#DEFINING A USER AGENT AND MAKING SURE STATUS IS GOOD TO GO
headers = {"User-agent" : "Mayuresh Bhangale"}
res = requests.get(url_1, headers=headers)
res.status_code

200

In [4]:
#PEEKING AT WHAT OUR DATA WILL LOOK LIKE
depress_json = res.json()
depress_json

{'kind': 'Listing',
 'data': {'after': 't3_1kg24ab',
  'dist': 27,
  'modhash': '',
  'geo_filter': None,
  'children': [{'kind': 't3',
    'data': {'approved_at_utc': None,
     'subreddit': 'depression',
     'selftext': 'We understand that most people who reply immediately to an OP with an invitation to talk privately  mean only to help, but this type of response usually leads to either disappointment or disaster.  it usually works out quite differently here than when you say "PM me anytime" in a casual social context.  \n\nWe have huge admiration and appreciation for the goodwill and good citizenship of so many of you who support others here and flag inappropriate content - even more so because we know that so many of you are struggling yourselves.  We\'re hard at work behind the scenes on more information and resources to make it easier to give and get quality help here - this is just a small start.  \n\nOur new wiki page explains in detail why it\'s much better to respond in publ

In [5]:
#THE REDDIT DATA SEEMS TO BE ORGANISED AS A DICTIONARY
#LET'S GET ITS KEYS
sorted(depress_json["data"].keys())

['after', 'before', 'children', 'dist', 'geo_filter', 'modhash']

In [6]:
#WE FIND OUT THAT THE after KEY IS THE QUERY STRING THAT WILL...
#INDICATE IN OUR URL THAT WE WANT TO SEE THE NEXT 25 POSTS AFTER THE after "CODE"

depress_json["data"]["after"]

't3_1kg24ab'

In [7]:
#DOUBLE CONFIRMING THAT THE PREVIOUS AFTER KEY IS REALLY THE LAST ITEM ON OUR PAGE
[post["data"]["name"] for post in depress_json["data"]["children"]]

['t3_doqwow',
 't3_1jys8x9',
 't3_1kg2uc3',
 't3_1kg6guu',
 't3_1kg5qfa',
 't3_1kg9za5',
 't3_1kg4hna',
 't3_1kg6xv3',
 't3_1kg42ro',
 't3_1kg4nq9',
 't3_1kfx6ug',
 't3_1kg6c2c',
 't3_1kfg4h9',
 't3_1kgbhrx',
 't3_1kga8nk',
 't3_1kga422',
 't3_1kgbtnc',
 't3_1kfutca',
 't3_1kgaclr',
 't3_1kfmo5t',
 't3_1kg9uco',
 't3_1kg3805',
 't3_1kg962u',
 't3_1kgcg0s',
 't3_1kgccm8',
 't3_1kg65h6',
 't3_1kg24ab']

In [8]:
#CHECKING OUT THE NUMBER OF POSTS IN ONE PAGE
len(depress_json["data"]["children"])

27

In [9]:
pd.DataFrame(depress_json["data"]["children"])

,kind,data
0,t3,"{'approved_at_utc': None, 'subreddit': 'depres..."
1,t3,"{'approved_at_utc': None, 'subreddit': 'depres..."
2,t3,"{'approved_at_utc': None, 'subreddit': 'depres..."
3,t3,"{'approved_at_utc': None, 'subreddit': 'depres..."
4,t3,"{'approved_at_utc': None, 'subreddit': 'depres..."
5,t3,"{'approved_at_utc': None, 'subreddit': 'depres..."
6,t3,"{'approved_at_utc': None, 'subreddit': 'depres..."
7,t3,"{'approved_at_utc': None, 'subreddit': 'depres..."
8,t3,"{'approved_at_utc': None, 'subreddit': 'depres..."
9,t3,"{'approved_at_utc': None, 'subreddit': 'depres..."


In [10]:
#THIS GIVES US THE DATA WE ARE LOOKING FOR
depress_json["data"]["children"][0]["data"]

{'approved_at_utc': None,
 'subreddit': 'depression',
 'selftext': 'We understand that most people who reply immediately to an OP with an invitation to talk privately  mean only to help, but this type of response usually leads to either disappointment or disaster.  it usually works out quite differently here than when you say "PM me anytime" in a casual social context.  \n\nWe have huge admiration and appreciation for the goodwill and good citizenship of so many of you who support others here and flag inappropriate content - even more so because we know that so many of you are struggling yourselves.  We\'re hard at work behind the scenes on more information and resources to make it easier to give and get quality help here - this is just a small start.  \n\nOur new wiki page explains in detail why it\'s much better to respond in public comments, at least until you\'ve gotten to know someone.  It will be maintained at /r/depression/wiki/private_contact, and the full text of the current v

### 1.2 Creating functions to automate the Data Collection process 
- We will first run those functions on r/depression and check if they have worked well.

In [11]:
# NOW WE CAN DEFINE A FUNCTION TO SCRAPE A REDDIT PAGE

def reddit_scrape(url_string, number_of_scrapes, output_list):
    #SCRAPED POSTS WILL BE CONTAINED IN OUTPUT LIST(SHD BE EMPTY)
    #THIS IS USEFUL FOR THE FIRST SCRAPE FROM THE VIRGIN SUBREDDIT
    after = None 
    for _ in range(number_of_scrapes):
        if _ == 0:
            print("SCRAPING {}\n--------------------------------------------------".format(url_string))
            print("<<<SCRAPING COMMENCED>>>") 
            print("Downloading Batch {} of {}...".format(1, number_of_scrapes))
        elif (_+1) % 5 ==0:
            print("Downloading Batch {} of {}...".format((_ + 1), number_of_scrapes))
        
        if after == None:
            params = {}
        else:
            #THIS WILL TELL THE SCRAPER TO GET THE NEXT SET AFTER REDDIT'S after CODE
            params = {"after": after}             
        res = requests.get(url_string, params=params, headers=headers)
        if res.status_code == 200:
            the_json = res.json()
            output_list.extend(the_json["data"]["children"])
            after = the_json["data"]["after"]
        else:
            print(res.status_code)
            break
        time.sleep(randint(1,6))
    
    print("<<<SCRAPING COMPLETED>>>")
    print("Number of posts downloaded: {}".format(len(output_list)))
    print("Number of unique posts: {}".format(len(set([p["data"]["name"] for p in output_list]))))
 

In [12]:
#CALLING THE FUNCTION ON OUR DEPRESSION SUBREDDIT
depress_scraped = [] #DEFINING AN EMPTY LIST THAT WILL CONTAIN OUR SCRAPED DATA
reddit_scrape("https://www.reddit.com/r/depression.json", 50, depress_scraped)

SCRAPING https://www.reddit.com/r/depression.json
--------------------------------------------------
<<<SCRAPING COMMENCED>>>
<<<SCRAPING COMPLETED>>>
Number of posts downloaded: 1252
Number of unique posts: 925


In [13]:
#CREATING A FUNCTION TO OUTPUT A LIST OF UNIQUE POSTS
def create_unique_list(original_scrape_list, new_list_name):
    data_name_list=[]
    for i in range(len(original_scrape_list)):
        if original_scrape_list[i]["data"]["name"] not in data_name_list:
            new_list_name.append(original_scrape_list[i]["data"])
            data_name_list.append(original_scrape_list[i]["data"]["name"])
    #CHECKING IF THE NEW LIST IS OF SAME LENGTH AS UNIQUE POSTS
    print("LIST NOW CONTAINS {} UNIQUE SCRAPED POSTS".format(len(new_list_name)))
    

In [14]:
#CALLING THE FUNCTION ON OUR SCRAPED DATA
depress_scraped_unique = []
create_unique_list(depress_scraped, depress_scraped_unique)

LIST NOW CONTAINS 925 UNIQUE SCRAPED POSTS


In [15]:
#PUTTING DEPRESSION DATA INTO A DATAFRAME AND SAVING TO CSV
depression = pd.DataFrame(depress_scraped_unique)
depression["is_suicide"] = 0
depression.head() #CHECK IF THERE ARE 100 COLS AND LAST DUMMY is_suicide COL

,approved_at_utc,subreddit,selftext,author_fullname,saved,mod_reason_title,gilded,clicked,title,link_flair_richtext,...,permalink,stickied,url,subreddit_subscribers,created_utc,num_crossposts,media,is_video,author_cakeday,is_suicide
0,None,depression,We understand that most people who reply immed...,t2_1t70,False,None,4,False,Our most-broken and least-understood rules is ...,[],...,/r/depression/comments/doqwow/our_mostbroken_a...,True,https://www.reddit.com/r/depression/comments/d...,1081886,1.572361e+09,2,None,False,NaN,0
1,None,depression,Welcome to /r/depression's check-in post - a p...,t2_1t70,False,None,0,False,"Regular check-in post, with essential informat...",[],...,/r/depression/comments/1jys8x9/regular_checkin...,True,https://www.reddit.com/r/depression/comments/1...,1081886,1.744612e+09,0,None,False,NaN,0
2,None,depression,Every day I wake up I have that thought. I wis...,t2_1gvkfazk1f,False,None,0,False,I wish I was dead,[],...,/r/depression/comments/1kg2uc3/i_wish_i_was_dead/,False,https://www.reddit.com/r/depression/comments/1...,1081886,1.746533e+09,0,None,False,NaN,0
3,None,depression,I don't see my life going anywhere. I don't re...,t2_8lykm3en,False,None,0,False,I am planning my death. Suicidal thoughts are ...,[],...,/r/depression/comments/1kg6guu/i_am_planning_m...,False,https://www.reddit.com/r/depression/comments/1...,1081886,1.746543e+09,0,None,False,NaN,0
4,None,depression,Me and my girlfriend are both 18 and we met a ...,t2_10gwwinsx6,False,None,0,False,Losing my girlfriend due to culture,[],...,/r/depression/comments/1kg5qfa/losing_my_girlf...,False,https://www.reddit.com/r/depression/comments/1...,1081886,1.746541e+09,0,None,False,NaN,0


### 1.3 Running our functions on the r/SuicideWatch subreddit 

In [16]:
#CALLING THE SCRAPING FUNCTION ON OUR SUICIDEWATCH SUBREDDIT
suicide_scraped = [] #DEFINING AN EMPTY LIST THAT WILL CONTAIN OUR SCRAPED DATA
reddit_scrape("https://www.reddit.com/r/SuicideWatch.json", 50, suicide_scraped)

SCRAPING https://www.reddit.com/r/SuicideWatch.json
--------------------------------------------------
<<<SCRAPING COMMENCED>>>
429
<<<SCRAPING COMPLETED>>>
Number of posts downloaded: 1221
Number of unique posts: 970


In [17]:
#CALLING THE "UNIQUE ONLY" FUNCTION ON OUR SCRAPED DATA
suicide_scraped_unique = []
create_unique_list(suicide_scraped, suicide_scraped_unique)

LIST NOW CONTAINS 970 UNIQUE SCRAPED POSTS


In [18]:
#PUTTING SUICIDEWATCH DATA INTO A DATAFRAME AND SAVING TO CSV
suicide_watch = pd.DataFrame(suicide_scraped_unique)
suicide_watch["is_suicide"] = 1
suicide_watch.head() #CHECK IF THERE ARE 100 COLS AND LAST DUMMY is_suicide COL

,approved_at_utc,subreddit,selftext,author_fullname,saved,mod_reason_title,gilded,clicked,title,link_flair_richtext,...,permalink,stickied,url,subreddit_subscribers,created_utc,num_crossposts,media,is_video,author_cakeday,is_suicide
0,None,SuicideWatch,We've been seeing a worrying increase in pro-s...,t2_1t70,False,None,1,False,New wiki on how to avoid accidentally encourag...,[],...,/r/SuicideWatch/comments/cz6nfd/new_wiki_on_ho...,True,https://www.reddit.com/r/SuicideWatch/comments...,524702,1.567526e+09,0,None,False,NaN,1
1,None,SuicideWatch,"Activism, i.e. advocating or fundraising for s...",t2_1t70,False,None,0,False,Please remember that NO ACTIVISM of any kind i...,[],...,/r/SuicideWatch/comments/pl9suy/please_remembe...,True,https://www.reddit.com/r/SuicideWatch/comments...,524702,1.631232e+09,0,None,False,NaN,1
2,None,SuicideWatch,"Fuck u, fuck u. Everyone who was depressed, su...",t2_1nl8sd16xi,False,None,0,False,"""if you're actually suicidal, you would did it...",[],...,/r/SuicideWatch/comments/1kg1v5a/if_youre_actu...,False,https://www.reddit.com/r/SuicideWatch/comments...,524702,1.746530e+09,0,None,False,NaN,1
3,None,SuicideWatch,I have been feeling deeply depressed/idealizin...,t2_1gd961sz99,False,None,0,False,Left my flat with the intention of killing my ...,[],...,/r/SuicideWatch/comments/1kfzajy/left_my_flat_...,False,https://www.reddit.com/r/SuicideWatch/comments...,524702,1.746520e+09,0,None,False,NaN,1
4,None,SuicideWatch,If I could just walk into a hospital and give ...,t2_1bmokjrfwq,False,None,0,False,I wish I could give my life to someone who wan...,[],...,/r/SuicideWatch/comments/1kg1jib/i_wish_i_coul...,False,https://www.reddit.com/r/SuicideWatch/comments...,524702,1.746529e+09,0,None,False,NaN,1


In [19]:
#suicide_watch.to_csv('../data/suicide_watch.csv', index = False)
#depression.to_csv('../data/depression.csv', index = False)

In [20]:
#INVESTIGATING THE CASE OF r/SuicideWatch HAVING AN ADDITIONAL COLUMN
suicide_watch.columns.difference(depression.columns)

Index([], dtype='object')

In [21]:
#LOOKING INTO THAT ADDITIONAL COLUMN
suicide_watch['author_cakeday'].isnull().value_counts()

author_cakeday
True     969
False      1
Name: count, dtype: int64